# Set parameters

In [1]:
input_fld = './'
weight_file = 'sensor_weights_and_model.hdf5'
num_output = 1
write_graph_def_ascii_flag = True
prefix_output_node_names_of_final_network = 'output_node'
output_graph_name = 'constant_graph_weights.pb'

In [2]:
from keras.models import Model
model = Model()
model.load_weights(weight_file)
model.summary()

Using TensorFlow backend.


TypeError: __init__() missing 2 required positional arguments: 'input' and 'output'

# initialize

In [2]:
from keras.models import load_model
import tensorflow as tf
import os
import os.path as osp
from keras import backend as K

output_fld = input_fld + 'tensorflow_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
weight_file_path = osp.join(input_fld, weight_file)

Using TensorFlow backend.


# Load keras model and rename output

In [3]:
K.set_learning_phase(0)
net_model = load_model(weight_file_path)

pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(net_model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

output nodes names are:  ['output_node0']


#### [optional] write graph definition in ascii

In [4]:
sess = K.get_session()

if write_graph_def_ascii_flag:
    f = 'only_the_graph_def.pb.ascii'
    tf.train.write_graph(sess.graph.as_graph_def(), output_fld, f, as_text=True)
    print('saved the graph definition in ascii format at: ', osp.join(output_fld, f))

saved the graph definition in ascii format at:  ./tensorflow_model/only_the_graph_def.pb.ascii


#### convert variables to constants and save

In [5]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))

INFO:tensorflow:Froze 5 variables.
Converted 5 variables to const ops.
saved the constant graph (ready for inference) at:  ./tensorflow_model/constant_graph_weights.pb
